In [1]:
%matplotlib inline
import os
from matplotlib import pyplot
from PIL import Image
from trainer import model, task
from tensorflow.python.tools import inspect_checkpoint
import tensorflow as tf
import numpy as np

In [2]:
img = 'data/img/img_1900.jpeg'
job_dir = 'lr_0.03__32_64_128__0.5_adam'

In [7]:
CKPT_DIR = './ckpt/'
MODEL = '32_64_128'
JOB_DIR = CKPT_DIR + MODEL
latest = tf.train.latest_checkpoint(JOB_DIR)

In [8]:
task.build_and_run_exports(latest, JOB_DIR, '1', [32, 64, 128])

INFO:tensorflow:Restoring parameters from ./ckpt/32_64_128/model.ckpt-18039
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./ckpt/32_64_128/export/1/saved_model.pb


In [11]:

def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename) as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

In [12]:
graph = load_graph('./ckpt/32_64_128/export/1/saved_model.pb')

DecodeError: Truncated message.

In [17]:
files = [f for f in os.listdir('./ckpt/32_64_128')]
for f in files:
    print f

checkpoint
model.ckpt-18039.data-00000-of-00001
model.ckpt-18039.index
model.ckpt-18039.meta


In [27]:
inspect_checkpoint.print_tensors_in_checkpoint_file(CKPT, None, True)

tensor_name:  conv_1/weights
[[[[-0.10094063  0.05364406 -0.10023373 ..., -0.06134745 -0.06306113
    -0.08636754]
   [-0.01865342 -0.06538481 -0.08138065 ..., -0.00062486 -0.00618994
     0.03107214]
   [-0.03428696 -0.0070802  -0.09090862 ...,  0.03994314 -0.08107346
    -0.08900052]
   ..., 
   [ 0.02503841 -0.01130346 -0.00148153 ...,  0.02100536  0.03157293
    -0.02490692]
   [ 0.06584738 -0.09676826 -0.04359005 ..., -0.08052327 -0.00047755
     0.01223763]
   [-0.00531509  0.05301152 -0.08284718 ...,  0.01559914 -0.01281296
    -0.04157673]]

  [[-0.09614827 -0.07459401  0.05643468 ..., -0.02683915 -0.04338992
    -0.02623689]
   [ 0.04828455 -0.10333441 -0.06058737 ..., -0.06128605 -0.07830374
    -0.10835075]
   [ 0.02851597  0.0349384  -0.01899897 ..., -0.0037969  -0.07982308
    -0.10372863]
   ..., 
   [-0.04744531 -0.0821493   0.01587281 ..., -0.09303276  0.05409895
    -0.01454657]
   [ 0.02484972  0.0181983   0.03315728 ..., -0.07352416 -0.08593672
     0.04494506]
   [ 

tensor_name:  conv_1/biases/Adam
[  1.65346842e-06   5.76343097e-12   3.99146778e-08  -5.94204448e-06
  -9.03545239e-09   4.87053399e-07  -4.16232893e-07  -3.05940716e-07
   1.23009812e-07   5.99879058e-05   1.11856600e-04  -1.94757667e-05
  -8.25941097e-05  -9.48811074e-09   0.00000000e+00   8.25365059e-05
   1.69440330e-06  -2.89354625e-06  -1.50666949e-06  -8.13849255e-08
   2.31394260e-16  -1.16581883e-04  -7.47332524e-05   1.77304162e-08
   1.23148931e-08   2.24529776e-06   8.45073009e-05  -4.85025211e-08
  -2.68999791e-08   2.23572238e-06   5.71906867e-07  -2.63683381e-04
   9.32006913e-07   3.43375495e-10   0.00000000e+00   3.39184393e-04
   1.15916288e-09  -2.59267413e-10   2.49014585e-04  -4.31271001e-05
   5.40499750e-04  -2.09917701e-07   9.29240684e-08   7.68996671e-08
   2.50484682e-05   0.00000000e+00   1.67868551e-04   1.77885404e-08
   1.23658669e-04  -9.00075861e-07  -6.52323652e-05  -8.91501895e-09
   3.19697196e-04  -8.15165748e-08   6.21751906e-06   1.75398952e-07
 

In [30]:
def plot_sample(x, y, axis):
    """
    Plots a single sample image with keypoints on top.
    
    Parameters
    ----------
    x     : 
            Image data.
    y     : 
            Keypoints to plot.
    axis  :
            Plot over which to draw the sample.   
    """
    axis.scatter(y[0::2] * 48 + 48, y[1::2] * 48 + 48, marker='x', s=50, color='r')

In [31]:
graph = tf.Graph()
latest_checkpoint = tf.train.latest_checkpoint(job_dir)
with graph.as_default():
    pth = tf.placeholder(tf.string)
    img_p = tf.image.decode_jpeg(tf.read_file(pth), 1)
    img_p = tf.to_float(img_p) / 255.
    img_p = tf.reshape(img_p, [1, 96, 96, 1])
    ev = model.model_fn(
        model.PREDICT,
        img_p,
        None,
        convs=[32,64,128]
    )
    saver = tf.train.Saver()
with tf.Session(graph=graph) as session:
    session.run([tf.local_variables_initializer(), tf.tables_initializer()])
    saver.restore(session, latest_checkpoint)
    u = session.run(ev.values(), feed_dict={pth: img})

INFO:tensorflow:Restoring parameters from lr_0.03__32_64_128__0.5_adam/model.ckpt-14443


In [1]:
fig = pyplot.figure(figsize=(10, 10))
my_data = np.reshape(u[0], (30))
ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
p = pyplot.imshow(Image.open(img), cmap='gray', )
plot_sample(Image.open(img), my_data, ax)
pyplot.show()

NameError: name 'pyplot' is not defined

In [73]:
my_data

array([ 0.35271794, -0.20726182, -0.36451191, -0.22964424,  0.20045209,
       -0.18499956,  0.51917905, -0.18876155, -0.20793559, -0.20571825,
       -0.53966409, -0.2085416 ,  0.15971792, -0.39055014,  0.66376549,
       -0.38879472, -0.14049494, -0.39685667, -0.65779382, -0.40349984,
        0.01867865,  0.18821128,  0.28768474,  0.59483111, -0.36480802,
        0.58380306, -0.02063689,  0.52422237, -0.02090274,  0.7248044 ], dtype=float32)